In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import pathlib

import torch

import hyclib as lib
import hyclib.core.configurable as conf

In [3]:
pathlib.Path('/tmp/pytest-of-hc3190/pytest-27/test_dict_save_load0/config.pkl').is_file()

False

In [121]:
class SubModule(conf.Configurable, torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.W = torch.nn.Parameter(torch.ones(2,2))
        self.x = conf.Parameter(dtype=int)
        self.a = conf.Parameter([0,1,2.1])
        
class Module(conf.Configurable, torch.nn.Module):
    def __init__(self, y, z):
        super().__init__()
        self.sub_module = SubModule()
        self.y = y
        self.z = z

m = Module(conf.Parameter([1,2]), torch.nn.Parameter(torch.zeros(2,2)))
config_dict = m.config_dict()
print(config_dict)
config_dict['sub_module.a'][0] = 5.5
m.load_config_dict(config_dict)
config_dict = m.config_dict()
print(config_dict)
print(m.y)

{'y': [1, 2], 'sub_module.x': 0, 'sub_module.a': [0, 1, 2.1]}
{'y': [1, 2], 'sub_module.x': 0, 'sub_module.a': [5.5, 1, 2.1]}
[1, 2]


In [134]:
hi = torch.zeros((5,0))
print(hi)

tensor([], size=(5, 0))


In [131]:
class Grid(conf.Configurable, torch.Tensor):
    def __new__(cls, *args, data=None, require_grads=False, **kwargs):
        # Reference: https://discuss.pytorch.org/t/subclassing-torch-longtensor/100377/3
        if data is None:
            data = get_grid(*args, **kwargs)
        return torch.Tensor._make_subclass(cls, data, require_grads) 
    
    def __init__(self, extents, shape=None, dxs=None, w_dims=None, **kwargs):
        super().__init__()
        
        if w_dims is None:
            w_dims = []

        self._extents = conf.Parameter(extentse)
        self._w_dims = conf.Parameter(w_dims)
        
    @property
    def tensor(self):
        return self.as_subclass(torch.Tensor) # return a pure torch.Tensor without all the extra attribute
    
    @property
    def extents(self):
        return self._extents
    
    @property
    def w_dims(self):
        return self._w_dims
    
def get_grid(extents, shape=None, dxs=None, w_dims=None, method='linspace', device='cpu'):
    """
    Get a grid (i.e. a tensor with shape (n_1, n_2, ..., n_N, n_1+...+n_N) where grid[*idx,:] are coordinates)
    by specifying the extents in each dimension and the shape (n_1, ..., n_N).
    
    Parameters:
        extents: list of tuples of scalars (2,) or list of scalars - If list of tuples, each i-th tuple
                 indicates the lower and upper bound of the i-th dimension. If list of scalars, the lower
                 and upper bound are interpreted as (-scalar/2, scalar/2) for method='linspace' and
                 (0, scalar) for method='arange'.
        shape: tuple - shape of the grid, ignored if method is not 'linspace'
        dxs: tuple - step sizes in each dimension, ignored if method is not 'arange'. If None, defaults to
             step sizes of 1 along each dimension.
        w_dims: list of ints - a list of the dimensions along which the endpoint is not included, which is
                useful for dimensions which are periodic/wrapped (w in w_dims stands for wrapped). Ignored
                if method='arange'.
        method: 'linspace' or 'arange'. Specifies whether torch.linspace or torch.arange is used.
        device: The device on which the grid is created.
        
    Returns:
        grid: torch.Tensor
        
    """
    assert all([len(extent) == 2 for extent in extents if isinstance(extent, tuple)])
    if method == 'linspace':
        if shape is None:
            raise ValueError("shape must be provided when method='linspace'")
        else:
            assert len(extents) == len(shape)
    if method == 'arange':
        if dxs is None:
            dxs = [1 for _ in range(len(extents))]
        else:
            assert len(extents) == len(dxs)
    if w_dims is None:
        w_dims = []
    else:
        assert isinstance(w_dims, list)
    if len(w_dims) > 0:
        assert max(w_dims) < len(extents)
    
    if method == 'linspace':
        extents = [extent if isinstance(extent, tuple) else (-extent/2, extent/2) for extent in extents]
        endpoints = [False if i in w_dims else True for i in range(len(extents))]
        grids_per_dim = [linspace(extent[0],extent[1],N,endpoint,device=device) for extent, N, endpoint in zip(extents, shape, endpoints)]
    elif method == 'arange':
        extents = [extent if isinstance(extent, tuple) else (0, extent) for extent in extents]
        grids_per_dim = [torch.arange(extent[0],extent[1],dx,device=device) for extent, dx in zip(extents, dxs)]
    else:
        raise NotImplementedError()
        
    grid = torch.stack(torch.meshgrid(*grids_per_dim, indexing='ij'), dim=-1)
    return grid

def linspace(start, end, steps, endpoint=True, **kwargs):
    if endpoint:
        return torch.linspace(start, end, steps, **kwargs)
    else:
        return torch.linspace(start, end, steps+1, **kwargs)[:-1] # exclude endpoint

In [133]:
grid = Grid([5.0,5.0],shape=(5,5))
print(grid.config_dict())

{'_extents': [5.0, 5.0], '_w_dims': []}


In [115]:
class A:
    def __init__(self):
        print("HERE")
        self.x = 1
    
    def __setattr__(self, name, value):
        print("in super")
        super().__setattr__(name, value)
        
class B(A):
    def __setattr__(self, name, value):
        print("in child")
        super().__setattr__(name, value)
        
b = B()

HERE
in child
in super


In [119]:
class A(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.W = torch.nn.Linear(2,2)
        self.x = torch.nn.Parameter(torch.tensor(2.0))
        
a = A()
# a.W = 3.0
a.x = 6.0

TypeError: cannot assign 'float' as parameter 'x' (torch.nn.Parameter or None expected)

In [67]:
hi = conf.Parameter(torch.tensor(10))
print(hi.data, hi.dtype)

InvalidConfigParameter: data 10 is not json serializable, try converting it to a compatible object type. If not possible, you can set is_jsonable=False, but then you can't save the config dict as json.

In [72]:
class Hi(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.x = torch.nn.Linear(2,2)
        
hi = Hi()
state_dict = hi.state_dict()
print(state_dict)
state_dict['x.weight'][0,1] = 1.0
print(hi.x.weight)
hi.load_state_dict(state_dict)
print(hi.x.weight)
state_dict['x.weight'][0,1] = -2.0
print(hi.x.weight)

OrderedDict([('x.weight', tensor([[ 0.5398, -0.6444],
        [ 0.4426,  0.5372]])), ('x.bias', tensor([-0.1677, -0.0337]))])
Parameter containing:
tensor([[0.5398, 1.0000],
        [0.4426, 0.5372]], requires_grad=True)
Parameter containing:
tensor([[0.5398, 1.0000],
        [0.4426, 0.5372]], requires_grad=True)
Parameter containing:
tensor([[ 0.5398, -2.0000],
        [ 0.4426,  0.5372]], requires_grad=True)


In [84]:
hi = {'a': 1}
print('a' in hi.keys())

True
